In [1]:
import pandas as pd

In [43]:
pd.options.display.max_columns = 22
pd.options.display.max_rows = 400

In [95]:
df_taxes = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/AY2009 Real Property_Personal Property.csv')

In [96]:
df_home = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/fixed_home_v02.csv')
df_cdbg = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/fixed_cdbg.csv')

In [94]:
df_utilities2012 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2012.csv')
df_utilities2013 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2013.csv')
df_utilities2014 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2014.csv')
df_utilities2015 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2015.csv')
df_utilities2016 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2016.csv')
df_utilities2017 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2017.csv')
df_utilities2018 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2018.csv')
df_utilities2019 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2019.csv')
df_utilities_list = [df_utilities2012,df_utilities2013,df_utilities2014,df_utilities2015,
                    df_utilities2016,df_utilities2017,df_utilities2018,df_utilities2019]
list1 = []
list2 = []
for df_u in df_utilities_list:
    list1.append(df_u['Premise Address'])
for series_u in list1:
    for row in series_u:
        list2.append(row)
s_utilities = pd.Series(pd.Series(list2).unique())

In [116]:
def genStrip():
    return lambda addr: addr.strip()

In [144]:
df_taxes['FULL_ADDRESS'] = df_taxes['HOUSE_NO'].astype(str).apply(genStrip()) + \
' ' + df_taxes["STREET_NAM"].apply(genStrip()) + ' ' + df_taxes["STREET_TYPE"].astype(str).apply(genStrip())

In [145]:
s_taxes = pd.Series(df_taxes['FULL_ADDRESS'].unique())

In [146]:
df_house = pd.concat([df_home, df_cdbg], ignore_index = True)
df_house.drop('Unnamed: 0', axis = 1, inplace = True)
s_house = pd.Series(df_house['Address'].unique())

In [147]:
s_utilities.shape

(30714,)

In [148]:
s_taxes.shape

(35098,)

In [149]:
s_house.shape

(670,)

In [150]:
s_house

0                        1707 EDGERLY AVE
1                        703 DARTMOUTH LN
2                      1115 W WADDELL AVE
3                        313 FORRESTAL LN
4                            1125 JOEL DR
5                          1519 MALONE DR
6                          719 ANDOVER LN
7                     305 S SHADOWLAWN DR
8                           1662 SUNNY LN
9                         511 MOULTRIE RD
10                          1100 SWIFT ST
11                   1009 E ROOSEVELT AVE
12                     1118 W WHITNEY AVE
13                           431 EBONY LN
14                           529 EBONY LN
15                           535 EBONY LN
16                           539 EBONY LN
17                           545 EBONY LN
18                           559 EBONY LN
19                           563 EBONY LN
20                           531 EBONY LN
21                           537 EBONY LN
22                           543 EBONY LN
23                           555 E

In [151]:
s_taxes

0            0 A C L RAILROAD nan
1         4408 A C L RAILROAD nan
2                   2700 ABBEY LN
3                   2701 ABBEY LN
4                   2702 ABBEY LN
5                   2703 ABBEY LN
6                   2704 ABBEY LN
7                   2705 ABBEY LN
8                   2706 ABBEY LN
9                   2707 ABBEY LN
10                  2708 ABBEY LN
11                  2710 ABBEY LN
12                  2712 ABBEY LN
13                  2714 ABBEY LN
14                  2715 ABBEY LN
15                  2716 ABBEY LN
16                  2717 ABBEY LN
17                  2718 ABBEY LN
18                  2719 ABBEY LN
19                  0 ABERDEEN RD
20               1900 ABERDEEN RD
21               1909 ABERDEEN RD
22                  1111 ACKER DR
23                  1112 ACKER DR
24                  1113 ACKER DR
25                  1114 ACKER DR
26                  1115 ACKER DR
27                  1116 ACKER DR
28                  1118 ACKER DR
29            

#### copy in cleaning code for each home and cdbg, then concatenate, then run.

In [155]:
s_house[~s_house.isin(s_taxes)]

2                      1115 W WADDELL AVE
7                     305 S SHADOWLAWN DR
8                           1662 SUNNY LN
11                   1009 E ROOSEVELT AVE
12                     1118 W WHITNEY AVE
25                         1802 SHARON DR
32                         514 EUGEMAR DR
37                   2230 S WASHINGTON ST
38                    1006 S JEFFERSON ST
42                        1212 S DAVIS ST
50                      1004 AMSTERDAM LN
54                      2620 S MADISON ST
55                    1410 S CAROLINA AVE
56                    1411 S CAROLINA AVE
57                    1412 S CAROLINA AVE
58                    1413 S CAROLINA AVE
59                    1414 S CAROLINA AVE
60                    1415 S CAROLINA AVE
61                    1416 S CAROLINA AVE
67                      504 S MAGNOLIA ST
69                     1110 W WADDELL AVE
70                          1605 SUNNY LN
71                          1609 SUNNY LN
72                         1407 E 